### Implementing Long Short-Term Memory to detect and classify Parkinsons' Freezing of Gait types in time series data

In [1]:
using Pkg

# Pkg.add("CSV")
# Pkg.add("NNlib")
# Pkg.add("DataFrames")
# Pkg.add("Distributions")
# Pkg.add("ResumableFunctions")
# Pkg.add("Flux")


In [2]:
using Flux
using Flux: @epochs, batch, throttle

using CSV
using NNlib
using DataFrames
using Distributions
using ResumableFunctions
using Random

In [3]:
parkinson = CSV.read("./filtered_data.csv", DataFrame)

Row,Id,Subject,Visit,Test,Medication,Time,AccV,AccML,AccAP,StartHesitation,Turn,Walking,event
,String15,String7,Int64,Int64,String3,Int64,Float64,Float64,Float64,Int64,Int64,Int64,String15
1,003f117e14,4dc2f8,3,2,on,0,-9.53394,0.566322,-1.41353,0,0,0,Normal
2,003f117e14,4dc2f8,3,2,on,1,-9.53614,0.564137,-1.44062,0,0,0,Normal
3,003f117e14,4dc2f8,3,2,on,2,-9.52935,0.561765,-1.42933,0,0,0,Normal
4,003f117e14,4dc2f8,3,2,on,3,-9.53124,0.564227,-1.41549,0,0,0,Normal
5,003f117e14,4dc2f8,3,2,on,4,-9.54082,0.561854,-1.42947,0,0,0,Normal
6,003f117e14,4dc2f8,3,2,on,5,-9.53658,0.552573,-1.41395,0,0,0,Normal
7,003f117e14,4dc2f8,3,2,on,6,-9.52954,0.548058,-1.4138,0,0,0,Normal
8,003f117e14,4dc2f8,3,2,on,7,-9.52449,0.552772,-1.41563,0,0,0,Normal
9,003f117e14,4dc2f8,3,2,on,8,-9.53577,0.55296,-1.42014,0,0,0,Normal


In [4]:
@resumable function data_loader(parkinson_dataframe, num_sequences, num_selected_sequences; labels=["StartHesitation", "Turn", "Walking", "Normal"])
    pdf = deepcopy(parkinson_dataframe)

    # Random generating sequences
    batch_size = div(size(pdf, 1), num_sequences)
    sequences = collect(1:num_sequences)
    shuffle!(sequences)
    
    # Sending to the model
    for i in 1:num_selected_sequences
        # if benne van a random generalt elemek tombunkben
        x = hcat(
            pdf[!, "AccV"][sequences[i] * batch_size : sequences[i] * batch_size + batch_size],
            pdf[!, "AccML"][sequences[i] * batch_size : sequences[i] * batch_size + batch_size],
            pdf[!, "AccAP"][sequences[i] * batch_size : sequences[i] * batch_size + batch_size]
        )
    
        y = Flux.onehotbatch(pdf[!, "event"][i:i+batch_size], labels)
        
        @yield x, y
    end
end

data_loader (generic function with 1 method)

In [5]:
function init_params(in::Integer, out::Integer; mean=0.0, std=1.0)
    [
        in, out,
        rand(Truncated(Normal(mean, std), -1, 1), out), # Wf
        rand(Truncated(Normal(mean, std), -1, 1), out), # Wi
        rand(Truncated(Normal(mean, std), -1, 1), out), # Wc
        rand(Truncated(Normal(mean, std), -1, 1), out), # Wo
        rand(Truncated(Normal(mean, std), -1, 1), out), # bf
        rand(Truncated(Normal(mean, std), -1, 1), out), # bi
        rand(Truncated(Normal(mean, std), -1, 1), out), # bc
        rand(Truncated(Normal(mean, std), -1, 1), out), # bo
        rand(Truncated(Normal(mean, std), -1, 1), out), # bo
        rand(Truncated(Normal(mean, std), -1, 1), out), # bo
        rand(Truncated(Normal(mean, std), -1, 1), out), # bo
        rand(Truncated(Normal(mean, std), -1, 1), out), # bo

        # both the Long-Term and Short-Term memories are initialized with 0 values
        zeros(out),  # c
        zeros(out),  # h
        true # update_memory
    ]
end


init_params (generic function with 1 method)

In [6]:
# implementing the forwarding method which is used in the Chaining process
function forward(x, lstm)
    for i in 1:size(x, 1):
        long_remember_percent = NNlib.sigmoid_fast((lstm.c * lstm.wlr1) + (x * lstm.wlr2) + lstm.blr1)
        potential_remember_percent = NNlib.sigmoid_fast((lstm.h * lstm.wpr1) + (x * lstm.wpr2) + lstm.bpr1)
        potential_memory = NNlib.tanh_fast((lstm.c * lstm.wp1) + (x * lstm.wp2) + lstm.bp1)
        updated_long_memory = (lstm.c * long_remember_percent) + (potential_memory * potential_remember_percent)
        output_percent = NNlib.sigmoid_fast((lstm.h * lstm.wo1) + (x * lstm.wo2) + lstm.bo1)
        updated_short_memory = NNlib.tanh_fast(updated_long_memory) * output_percent

        # updating the memory
        if lstm.update_memory
            lstm.c, lstm.h = updated_long_memory, updated_short_memory
        end
    end

    [updated_short_memory]
end

forward (generic function with 1 method)

In [7]:
# custom Long Short-Term Memory layer
mutable struct LSTM
    # input and output size of the layer
    in::Integer
    out::Integer

    wlr1::Vector{Float32}
    wlr2::Vector{Float32}
    blr1::Vector{Float32}

    wpr1::Vector{Float32}
    wpr2::Vector{Float32}
    bpr1::Vector{Float32}

    wp1::Vector{Float32}
    wp2::Vector{Float32}
    bp1::Vector{Float32}

    wo1::Vector{Float32}
    wo2::Vector{Float32}
    bo1::Vector{Float32}


    # cell state (aka. long-term memory) and hidden state (aka. short-term memory)
    c::Vector{Float32}
    h::Vector{Float32}

    # prevents the model from modifying the memory state in case of testing and loss calculation
    # it needs to be set explicitely before and after calling the model(x) funtion
    update_memory::Bool
end

In [8]:
# defining the constructor
LSTM(in::Integer, out::Integer) = LSTM(init_params(in, out)...)

LSTM

In [9]:
# Overload call, so the object can be used as a function
(lstm::LSTM)(x) = forward(x, lstm)

In [10]:
# creating a functor from the struct, so that the training can optimize its parameters
Flux.@functor LSTM

In [11]:
# creating the Long Short-Term Memory layer
function LSTM((in, out)::Pair)
    LSTM(in, out) # constructor
end

LSTM

In [12]:
# explicitely defining the trainable parameters of the layer
# all the Wrights and Biases are trainable
# exceptions >> Cell State and Hidden State
# Flux.trainable(lstm::LSTM) = (lstm.Wf, lstm.Wi, lstm.Wc, lstm.Wo, lstm.bf, lstm.bi, lstm.bc, lstm.bo,)
Flux.trainable(lstm::LSTM) = (lstm.wlr1, lstm.wlr2, lstm.blr1, lstm.wpr1, lstm.wpr2, lstm.bpr1, lstm.wp1, lstm.wp2, lstm.bp1, lstm.wo1, lstm.wo2, lstm.bo1)

In [13]:
input_size = 128
hidden_size = 1
num_classes = 4 

model = Chain(
    LSTM(input_size => hidden_size),
    Dense(hidden_size => num_classes),
    softmax
)

;

In [14]:
function loss(x, y)
    model[1].update_memory = false
    l = Flux.crossentropy(model(x), y)
    model[1].update_memory = true

    l
end

optimizer = ADAM(0.001)
epochs = 10
batch_size = 127 # data loader returns batch_size + 1 samples

;

In [15]:
for epoch in 1:epochs
  for (input, output) in [(1, [0,0,1,0]), (1, [0,0,1,0])] #data_loader(parkinson, batch_size)
    grads = Flux.gradient(Flux.params(model)) do
      loss(input, output)
    end
    Flux.update!(optimizer, Flux.params(model), grads)

    break
  end

  # break
end

In [16]:
model[1].update_memory = false
model(10)

4-element Vector{Float32}:
 0.2487438
 0.24874434
 0.2537681
 0.24874379